In [4]:
from datasets import load_dataset
import matplotlib.pyplot as plt

# Tải bộ dữ liệu từ Hugging Face Hub
dataset = load_dataset("natix-network-org/roadwork")
train_ds = dataset["train"]
test_ds = dataset["test"]

print(dataset)

DatasetDict({
    train: Dataset({
        features: ['image', 'id', 'width', 'height', 'license', 'flickr_url', 'coco_url', 'date_captured', 'gps', 'city_name', 'scene_description', 'video_info.frame_id', 'video_info.seq_id', 'video_info.vid_id', 'scene_level_tags.daytime', 'scene_level_tags.scene_environment', 'scene_level_tags.travel_alteration', 'scene_level_tags.weather', 'label'],
        num_rows: 6251
    })
    test: Dataset({
        features: ['image', 'id', 'width', 'height', 'license', 'flickr_url', 'coco_url', 'date_captured', 'gps', 'city_name', 'scene_description', 'video_info.frame_id', 'video_info.seq_id', 'video_info.vid_id', 'scene_level_tags.daytime', 'scene_level_tags.scene_environment', 'scene_level_tags.travel_alteration', 'scene_level_tags.weather', 'label'],
        num_rows: 2298
    })
})


In [5]:
'''
import matplotlib.pyplot as plt

# Duyệt và lấy mỗi lớp 1 ảnh đầu tiên
sample_label_0 = next(sample for sample in train_ds if sample["label"] == 0)
sample_label_1 = next(sample for sample in train_ds if sample["label"] == 1)

# Hiển thị ảnh có label = 0
print("Ảnh mẫu thuộc lớp 0:")
plt.imshow(sample_label_0["image"])
plt.axis("off")
plt.title("Label 0")
plt.show()

# Hiển thị ảnh có label = 1
print("Ảnh mẫu thuộc lớp 1:")
plt.imshow(sample_label_1["image"])
plt.axis("off")
plt.title("Label 1")
plt.show()
'''


'\nimport matplotlib.pyplot as plt\n\n# Duyệt và lấy mỗi lớp 1 ảnh đầu tiên\nsample_label_0 = next(sample for sample in train_ds if sample["label"] == 0)\nsample_label_1 = next(sample for sample in train_ds if sample["label"] == 1)\n\n# Hiển thị ảnh có label = 0\nprint("Ảnh mẫu thuộc lớp 0:")\nplt.imshow(sample_label_0["image"])\nplt.axis("off")\nplt.title("Label 0")\nplt.show()\n\n# Hiển thị ảnh có label = 1\nprint("Ảnh mẫu thuộc lớp 1:")\nplt.imshow(sample_label_1["image"])\nplt.axis("off")\nplt.title("Label 1")\nplt.show()\n'

In [6]:
from collections import Counter
from datasets import Dataset
import random
from torchvision import transforms
from PIL import Image
import torch

# 📊 Đếm nhãn
train_labels = [ex["label"] for ex in train_ds]
train_counts = Counter(train_labels)
print("📊 Phân phối nhãn ban đầu:", train_counts)

# 🔍 Thông tin lớp thiểu số
minority_label = min(train_counts, key=train_counts.get)
majority_count = max(train_counts.values())
minority_count = train_counts[minority_label]
num_to_generate = majority_count - minority_count
print(f"✅ Cần tạo thêm {num_to_generate} ảnh cho lớp {minority_label}")

# 📦 Tạo transform augment
augment_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
])

# 🌀 Tạo ảnh augment từng bước (tránh giữ tất cả ảnh gốc trong RAM)
augmented_data = []
generated = 0

for ex in train_ds:
    if ex["label"] == minority_label:
        img = ex["image"]
        if not isinstance(img, Image.Image):
            img = img.convert("RGB")

        aug_img = augment_transform(img)
        augmented_data.append({"image": aug_img, "label": minority_label})
        generated += 1

        if generated >= num_to_generate:
            break

print(f"✅ Đã tạo {len(augmented_data)} ảnh augment.")

from datasets import concatenate_datasets

# Tạo Dataset từ augmented_data
aug_ds = Dataset.from_list(augmented_data)

# Nối 2 dataset mà không cần load vào RAM
train_ds = concatenate_datasets([train_ds, aug_ds])

# ✅ Đã tạo {len(augmented_data)} ảnh augment.
from datasets import concatenate_datasets

# 🔁 Gộp lại dùng concatenate_datasets (tránh MemoryError)
aug_ds = Dataset.from_list(augmented_data)
train_ds = concatenate_datasets([train_ds, aug_ds])



📊 Phân phối nhãn ban đầu: Counter({1: 5031, 0: 1220})
✅ Cần tạo thêm 3811 ảnh cho lớp 0
✅ Đã tạo 1220 ảnh augment.


In [7]:
# đặt tên cho các lớp
label_names = {
    0: "No Roadwork",
    1: "Roadwork"
}


In [8]:
from transformers import ViTImageProcessor
from torchvision import transforms

# Dùng processor tương ứng với mô hình ViT
processor = ViTImageProcessor.from_pretrained("google/vit-base-patch16-224-in21k")


# Hàm tiền xử lý toàn bộ batch
def preprocess(batch):
    images = [img.resize((224, 224)) for img in batch["image"]]
    encoding = processor(images, return_tensors="pt")
    return {
        "pixel_values": encoding["pixel_values"],
        "labels": batch["label"]
    }

# Tiền xử lý tập huấn luyện và kiểm tra với batch_size nhỏ để tránh MemoryError
train_ds = train_ds.map(preprocess, batched=True, batch_size=16, remove_columns=["image"])
test_ds = test_ds.map(preprocess, batched=True, batch_size=16, remove_columns=["image"])

Map: 100%|██████████| 8691/8691 [10:27<00:00, 13.86 examples/s] 


In [9]:
''' 
from collections import Counter
import torch

# 🛠 Dummy transform không làm gì cả
def no_transform(example):
    return example

# 🔁 Gán tạm transform "trống"
train_ds.set_transform(no_transform)
test_ds.set_transform(no_transform)

# ✅ Lấy nhãn an toàn
train_labels = [example["label"] for example in train_ds]
test_labels = [example["label"] for example in test_ds]

# ✅ Gán lại transform gốc
train_ds.set_transform(transform)
test_ds.set_transform(transform)

# 📊 Đếm số lượng lớp
train_counts = Counter(train_labels)
test_counts = Counter(test_labels)

print("📊 Phân phối nhãn train:", train_counts)
print("📊 Phân phối nhãn test:", test_counts)

# ⚖️ Tính class weights
total = sum(train_counts.values())
class_weights = [total / train_counts[i] for i in sorted(train_counts.keys())]
class_weights = torch.tensor(class_weights, dtype=torch.float)

# ⚙️ Dùng CPU
device = torch.device("cpu")
class_weights = class_weights.to(device)

print("✅ class_weights =", class_weights)
'''

' \nfrom collections import Counter\nimport torch\n\n# 🛠 Dummy transform không làm gì cả\ndef no_transform(example):\n    return example\n\n# 🔁 Gán tạm transform "trống"\ntrain_ds.set_transform(no_transform)\ntest_ds.set_transform(no_transform)\n\n# ✅ Lấy nhãn an toàn\ntrain_labels = [example["label"] for example in train_ds]\ntest_labels = [example["label"] for example in test_ds]\n\n# ✅ Gán lại transform gốc\ntrain_ds.set_transform(transform)\ntest_ds.set_transform(transform)\n\n# 📊 Đếm số lượng lớp\ntrain_counts = Counter(train_labels)\ntest_counts = Counter(test_labels)\n\nprint("📊 Phân phối nhãn train:", train_counts)\nprint("📊 Phân phối nhãn test:", test_counts)\n\n# ⚖️ Tính class weights\ntotal = sum(train_counts.values())\nclass_weights = [total / train_counts[i] for i in sorted(train_counts.keys())]\nclass_weights = torch.tensor(class_weights, dtype=torch.float)\n\n# ⚙️ Dùng CPU\ndevice = torch.device("cpu")\nclass_weights = class_weights.to(device)\n\nprint("✅ class_weights =

In [10]:
from transformers import ViTForImageClassification

model = ViTForImageClassification.from_pretrained(
    "google/vit-base-patch16-224",
    num_labels=2,
    id2label={0: "No Roadwork", 1: "Roadwork"},
    label2id={"No Roadwork": 0, "Roadwork": 1},
    ignore_mismatched_sizes=True  
)


Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([2]) in the model instantiated
- classifier.weight: found shape torch.Size([1000, 768]) in the checkpoint and torch.Size([2, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
'''from transformers import AutoModelForImageClassification

model = AutoModelForImageClassification.from_pretrained(
    "google/vit-base-patch16-224",
    num_labels=2,  # Vì có 2 lớp
    id2label=label_names,  # Đặt lại tên lớp cho đẹp
    label2id={v: k for k, v in label_names.items()},
    ignore_mismatched_sizes=True  # Cho phép thay đổi output layer
)
'''

'from transformers import AutoModelForImageClassification\n\nmodel = AutoModelForImageClassification.from_pretrained(\n    "google/vit-base-patch16-224",\n    num_labels=2,  # Vì có 2 lớp\n    id2label=label_names,  # Đặt lại tên lớp cho đẹp\n    label2id={v: k for k, v in label_names.items()},\n    ignore_mismatched_sizes=True  # Cho phép thay đổi output layer\n)\n'

In [12]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="steps",  # Đúng với transformers >= 4.0.0
    eval_steps=500,
    save_steps=500,
)


In [13]:
from sklearn.metrics import accuracy_score, f1_score
import numpy as np

def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    labels = p.label_ids
    return {
        "accuracy": accuracy_score(labels, preds),
        "f1": f1_score(labels, preds, average="weighted")
    }


In [ ]:
from transformers import default_data_collator

def custom_data_collator(features):
    for feature in features:
        feature.pop("id", None)  # loại bỏ id
    return default_data_collator(features)


from transformers import Trainer

# Đảm bảo chỉ giữ lại các cột cần thiết (pixel_values, labels)
training_args.remove_unused_columns = True

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    compute_metrics=compute_metrics,
    data_collator=custom_data_collator  # <<< sửa ở đây
)

trainer.train()
trainer.evaluate()

Step,Training Loss,Validation Loss,Accuracy,F1
500,0.222700,0.127344,0.965187,0.962241
1000,0.202100,0.152190,0.965622,0.962760
1500,0.121100,0.121565,0.969974,0.969772
2000,0.090700,0.140679,0.966057,0.965662
2500,0.054500,0.147602,0.970844,0.969527


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# Dự đoán trên tập test (hoặc validation)
predictions = trainer.predict(test_ds)
y_pred = np.argmax(predictions.predictions, axis=1)
y_true = predictions.label_ids

# Tính confusion matrix
cm = confusion_matrix(y_true, y_pred)

# Vẽ confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix")
plt.show()


In [ ]:
from PIL import Image
import torch
from transformers import AutoImageProcessor, ViTForImageClassification

# Load mô hình và processor
# Nếu đã fine-tune xong rồi thì dùng dòng này để load lại:
# model = ViTForImageClassification.from_pretrained("output_model")
# processor = AutoImageProcessor.from_pretrained("google/vit-base-patch16-224")

# Thiết bị tính toán
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Ảnh người dùng
image_path = r"C:\Users\admin\Documents\tlh\ielts\task 1\test1.jpg"
image = Image.open(image_path).convert("RGB")
image.show()

# Tiền xử lý và đưa lên đúng thiết bị
inputs = processor(images=image, return_tensors="pt")
inputs = {k: v.to(device) for k, v in inputs.items()}

# Dự đoán
model.eval()
with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits
    predicted_class = torch.argmax(logits, dim=1).item()

# In kết quả
label = model.config.id2label[predicted_class]
print(f"Ảnh dự đoán thuộc lớp: {predicted_class} - {label}")
